In [2]:
from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
from datetime import datetime
from functools import partial

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))

In [3]:
srFile = "examples/CoDec_Research/results/simulation_results/server/Set2_construal_vals_2025-05-11 15_02_12.469384.pickle"
with open(srFile, 'rb') as opn_file:
    construal_action_likelihoods = pickle.load(opn_file)


In [4]:
construal_action_likelihoods

{'dict_structure': '{scene_name: {construal_index: value}}',
 'tfrecord-00035-of-01000_85': {(0,): 1.0,
  (1,): 1.0,
  (2,): 1.0,
  (3,): 1.0,
  (4,): 0.975,
  (5,): 1.0,
  (35,): 1.0,
  (41,): 1.0,
  (0, 1, 2, 3, 4, 5, 35, 41): 1.0},
 'tfrecord-00046-of-01000_167': {(0,): 1.0,
  (1,): 1.0,
  (2,): 1.0,
  (3,): 1.0,
  (4,): 1.0,
  (5,): 1.0,
  (6,): 1.0,
  (7,): 1.0,
  (8,): 1.0,
  (10,): 1.0,
  (11,): 1.0,
  (12,): 1.0,
  (15,): 1.0,
  (17,): 1.0,
  (18,): 1.0,
  (19,): 1.0,
  (20,): 1.0,
  (24,): 1.0,
  (31,): 1.0,
  (0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 15, 17, 18, 19, 20, 24, 31): 0.975},
 'tfrecord-00047-of-01000_484': {(0,): 1.0,
  (2,): 0.975,
  (3,): 1.0,
  (4,): 1.0,
  (5,): 1.0,
  (6,): 1.0,
  (7,): 1.0,
  (8,): 1.0,
  (9,): 1.0,
  (10,): 1.0,
  (11,): 1.0,
  (12,): 1.0,
  (13,): 1.0,
  (14,): 1.0,
  (16,): 1.0,
  (17,): 1.0,
  (18,): 1.0,
  (19,): 1.0,
  (21,): 1.0,
  (22,): 1.0,
  (23,): 1.0,
  (24,): 1.0,
  (26,): 1.0,
  (27,): 0.975,
  (28,): 1.0,
  (29,): 1.0,
  (30,):

In [1]:
"""
    This part of pipeline 1 is dedicated to synthetic data generation. The code generates synthetic 
    data based on sampled construals (previous stage of pipeline).
"""

from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
import gc
from datetime import datetime
from functools import partial

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.simulation.construal_main import generate_baseline_data, generate_selected_construal_traj, \
                                                                    get_constral_heurisrtic_values, generate_all_construal_trajnval
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars, save_pickle
from examples.CoDec_Research.code.config import active_config


# Function to extract filename from path
env_path2name = lambda path: path.split("/")[-1].split(".")[0]


# |START TIMER
start_time = time.perf_counter()

####################################################
################ SET EXP PARAMETERS ################
####################################################

curr_config = active_config

# Parameters for Inference
heuristic_params = {"ego_distance": 0.5, "cardinality": 1}              # Hueristics and their weight parameters (to be inferred)

construal_count_baseline = curr_config['construal_count_baseline']      # Number of construals to sample for baseline data generation
trajectory_count_baseline = curr_config['trajectory_count_baseline']    # Number of baseline trajectories to generate per construal


### Specify Environment Configuration ###

# |Location to store (and retrieve pre-computed) simulation results
simulation_results_path = "examples/CoDec_Research/results/simulation_results/"
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]

# |Model Config (on which model was trained)
training_config = load_config("examples/experimental/config/reliable_agents_params")

# |Set scenario path
dataset_path = 'data/processed/construal/Set3.1/'
processID = dataset_path.split('/')[-2]                 # Used for storing and retrieving relevant data

# |Set simulator config
max_agents = training_config.max_controlled_agents      # Get total vehicle count
num_parallel_envs = curr_config['num_parallel_envs']
total_envs = curr_config['total_envs']
device = eval(curr_config['device'])

# |Set construal config
construal_size = 1
observed_agents_count = max_agents - 1                              # Agents observed except self (used for vector sizes)
sample_size_utility = curr_config['sample_size_utility']            # Number of samples to compute expected utility of a construal

# |Other changes to variables
training_config.max_controlled_agents = 1                           # Control only the first vehicle in the environment
total_envs = min(total_envs, len(listdir(dataset_path)))


env_config, train_loader, env, env_multi_agent, sim_agent = get_gpuDrive_vars(
                                                                                training_config=training_config,
                                                                                device=device,
                                                                                num_parallel_envs=num_parallel_envs,
                                                                                dataset_path=dataset_path,
                                                                                max_agents=max_agents,
                                                                                total_envs=total_envs,
                                                                                sim_agent_path="daphne-cornelisse/policy_S10_000_02_27",
                                                                        )



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-12 00:59:48.945057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747025988.961097   39765 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747025988.965320   39765 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747025988.977305   39765 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747025988.977355   39765 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747025988.977356   39765 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [4]:


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default
import pickle

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))

In [5]:
srFile = "examples/CoDec_Research/results/simulation_results/Set3_baseline_state_action_pairs_2025-05-13 15:33:50.425015.pickle"
with open(srFile, 'rb') as opn_file:
    construal_action_likelihoods = pickle.load(opn_file)

In [20]:
construal_action_likelihoods['tfrecord-00035-of-01000_85'][(4,)][0][0][0][0][0]


tensor([ 0.2009,  0.1762,  0.1555,  0.1809, -0.0006,  0.0000])

In [62]:
construal_action_likelihoods['tfrecord-00035-of-01000_85'][(4,)][0][-30][0][0][0]

tensor([  0.0000,   0.1762,   0.1555,   7.3817, -13.8764,   0.0000])

# TEMP

In [33]:

srFile = "examples/CoDec_Research/results/simulation_results/Set3_baseline_state_action_pairs_2025-05-14 00:56:25.396045.pickle"
with open(srFile, 'rb') as opn_file:
    test_data = pickle.load(opn_file)

In [34]:
test_data.keys()


dict_keys(['dict_structure', 'tfrecord-00101-of-01000_71', 'tfrecord-00183-of-01000_301', 'params'])

In [35]:
test_data['tfrecord-00101-of-01000_71'][(0,)].keys()

dict_keys([0, 1, 2])

In [ ]:
srFile = "/mnt/d/Data/Set3_baseline_state_action_pairs_2025-05-13 15:33:50.425015.pickle"
with open(srFile, 'rb') as opn_file:
    test_data = pickle.load(opn_file)

In [ ]:
srFile = "examples/CoDec_Research/results/simulation_results/Set3_construal_action_likelihoods_ego_distance0.5_cardinality1.tsv"
with open(srFile, 'rb') as opn_file:
    test_data = pickle.load(opn_file)